# API Asos

In [1]:
import json
import requests
import pandas as pd
import os
import sys
import dotenv

dotenv.load_dotenv()

sys.path.append(os.path.abspath('../'))
from src import funciones_auxiliares as fa
from src import dic_colores

Buscamos en la API de Asos vestidos de Mango, ya que es una de las marcas favoritas de las clientas:

In [ ]:
url = "https://asos10.p.rapidapi.com/api/v1/getProductList"
id_mango = 8264
api_key = os.getenv("api_asos")

querystring = {"categoryId":id_mango, "sort":"recommended", "limit":2000}

headers = {
	"x-rapidapi-key": api_key,
	"x-rapidapi-host": "asos10.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
response_mango = response.json()

# Guardamos el resultado en datos
with open("../datos/api_asos/vestidos_mango.json", 'w') as file:
    json.dump(response_mango, file, indent=4)


Filtramos los productos que sean vestidos:

In [2]:
lista_id_vestido = []
dic_precio = {}
with open(f"../datos/api_asos/vestidos_mango.json", 'r') as file:
    response_mango = json.load(file)

for i in range(len(response_mango["data"]["products"])):
    if "dress" in response_mango["data"]["products"][i]["name"]:
        id = response_mango["data"]["products"][i]["id"]
        lista_id_vestido.append(id)
        dic_precio[id]=response_mango["data"]["products"][i]["price"]["current"]["value"]

Para cada vestido volvemos a llamar a la api para que nos de aún más información de cada producto.

In [ ]:
api_key = os.getenv("api_asos")

for id in lista_id_vestido:

    url = "https://asos10.p.rapidapi.com/api/v1/getProductDetails"

    querystring = {"productId":[id]}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "asos10.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    res = response.json()
    with open(f"../datos/api_asos/vestido_{id}.json", 'w') as file:
        json.dump(res, file, indent=4)

Generamos un diccionario limpio con todos los valores necesarios y creamos un daraframe.

In [13]:
dic_vestidos = fa.creacion_diccionario_asos(dic_precio)
df_mango = pd.DataFrame(dic_vestidos)

In [14]:
# cambiamos los colores a español
df_mango["color"] = df_mango["color"].map(dic_colores.colores)

# añadimos la columna de categoria del vestido
df_mango["nombre"] = df_mango["nombre"].str.lower()
df_mango["categoria"] = df_mango["nombre"].str.extract('(largo|midi|corto|maxi|mini)', expand=True)

Una vez comprobado que no hay duplicados y nulos solo en la categoria creada por nosotros, procedemos a almacenar el dataframe.

In [15]:
df_mango.to_csv("../datos/dataframes/df_mango.csv")

In [12]:
df_mango.head(3)

,nombre,marca,precio,color,talla,disponible,categoria
0,vestido midi plateado con cuello halter de len...,Mango,133.0,plata,S,True,midi
1,vestido midi plateado con cuello halter de len...,Mango,133.0,plata,M,True,midi
2,vestido midi plateado con cuello halter de len...,Mango,133.0,plata,L,True,midi
